# Cuaderno 19: Problema del agente viajero múltiple ($k$-ATSP) 
$\newcommand{\ZZ}{\mathbb{Z}}$
Dados: 
* un grafo dirigido **completo** $D=(V,A)$, con $V=\{0,1,\ldots,n\}$; 
* un número entero $k$; y,
* un vector $c \in \ZZ^{A}$ de costos asociados a los arcos de $D$.

El *problema del agente viajero asimétrico múltiple (Multiple Asymmetric Traveling Salesman Problem, $k$-ATSP)* consiste en encontrar un conjunto de $k$ circuitos dirigidos (también llamados **rutas**) con las siguientes propiedades:
* todas las rutas deben contener al nodo 0 (también conocido como *nodo base* o *nodo depósito*);
* cada uno de los demás nodos $1, \ldots, n$ (conocidos como *nodos clientes*) debe pertenecer exactamente a una ruta; y,
* la suma de los costos de los arcos de todas las rutas debe ser mínima.


Aplicaciones de este problema se presentan en la logística de distribución, particularmente en el enrutamiento óptimo de flotas de vehículos. El problema puede formularse como un programa lineal entero, extendiendo cualquiera de las formulaciones del ATSP presentadas en los cuadernos anteriores. A continuación, presentamos un modelo basado en la *formulación compacta MTZ (Miller-Tucker-Zemlin)*. Emplearemos las siguientes variables de decisión:

* variables binarias $x_{ij}$, para indicar si los arcos de $A$ son seleccionados o no dentro de alguna ruta; y
* variables enteras $u_i$, para indicar la posición del nodo $i$ dentro de su ruta.

El problema puede ser formulado como el siguiente programa lineal entero:

\begin{align*}
\min &\sum_{(i,j) \in A} c_{ij} x_{ij}\\ 
& \mbox{s.r.}\\
&\sum_{(i,j) \in \delta^+(i) } x_{ij} = 1, \quad \forall i \in V \setminus \{0\},\\
&\sum_{(j,i) \in \delta^-(i) } x_{ji} = 1, \quad \forall i \in V \setminus \{0\},\\
&\sum_{(0,j) \in \delta^+(0) } x_{0j} = k,\\
&\sum_{(j,0) \in \delta^-(0) } x_{j0} = k,\\
& u_j \geq u_i + 1- (1 - x_{ij})n , \quad\forall (i,j) \in A, i \neq 0, j \neq 0, \\
& x_{ij} \in \{0, 1\}, \quad \forall (i, j) \in A,\\
& u_i \in \{1, \ldots, n\}, \quad \forall i \in A \setminus \{0\}.
\end{align*}

La función objetivo mide el costo total de todas las rutas seleccionadas.

Las dos primeras familias de restricciones son restricciones de grado para los nodos clientes: especifican que para cada uno de estos nodos se deben seleccionar un arco entrante y un arco saliente en la solución.

La tercera y cuarta restricciones son restricciones de grado del depósito. Indican que deben seleccionarse exactamente $k$ arcos salientes y $k$ arcos entrantes al depósito. 

Finalmente, la quinta familia de restricciones emplea la idea del modelo MTZ del agente viajero para eliminar circuitos que no contengan al depósito.

Vamos a implementar este modelo utilizando el API Python de Gurobi.


In [ ]:
import gurobipy as gp
from gurobipy import GRB
import random as rm
import math 

# número de nodos del grafo
n = 40

# número de rutas en la solución
k = 4

# nodos del grafo
V = gp.tuplelist(range(0,n+1))

# nodos sin el depósito
N = gp.tuplelist([i for i in V if i!=0])

# posiciones de los nodos en un plano euclideano entre (0,0) y (100, 100)
coordx={i : rm.randint(0,100) for i in N}
coordy={i : rm.randint(0,100) for i in N}
# el depósito está en el centro
coordx[0] = 50
coordy[0] = 50

# los costos son las distancias eculideanas
c = gp.tupledict({
    (i,j) : math.sqrt((coordx[i] - coordx[j])**2 + (coordy[i] - coordy[j])**2)
    for i in V for j in V if i!=j
})

A = c.keys()


### Implementación del modelo

Definimos el objeto modelo, las variables y la función objetivo. 

In [ ]:
# crear el objeto modelo
m = gp.Model('k-atsp')

# crear las variables de selección de arcos
x = m.addVars(A, name="x", vtype=GRB.BINARY)

# crear las variables de ordenamiento de nodos
u = m.addVars(N, name="u", lb=1, ub=n)

# crear la función objetivo
m.setObjective(x.prod(c,'*'), GRB.MINIMIZE)

Añadimos las restricciones de grado en los nodos cliente:

In [ ]:
# restricciones de grado saliente clientes
m.addConstrs((x.sum(i,'*') == 1 for i in N), "g_saliente")

# restricciones de grado entrante clientes
m.addConstrs((x.sum('*', i) == 1 for i in N), "g_entrante")

Agregamos las restricciones de grado entrante y saliente en el depósito:

In [ ]:
# restricción de grado saliente depósito
m.addConstr(x.sum(0,'*') == k, "d_saliente")

# restricción de grado entrante depósito
m.addConstr(x.sum('*', 0) == k, "d_entrante")

Definimos las restricciones de eliminación de subciclos:

In [ ]:
# restricciones de ordenamiento de nodos (eliminación de subciclos)
m.addConstrs((u[j] >= u[i] + 1 - n*(1- x[i,j]) for i in N for j in N if i!=j), 
             "ordenamiento")

Fijamos un límite para el tiempo de cálculo y una tolerancia para la brecha de optimalidad:

In [ ]:
# terminar al alcanzar un Gap del 10%
m.Params.MIPGap = 0.1

# terminar luego de 180 segundos
m.Params.TimeLimit = 180

Finalmente, resolvemos el modelo y mostramos la solución, en caso de que se haya encontrado una solución factible (si `m.SolCount` es mayor a cero).

In [ ]:
# resolver el modelo
m.optimize()

# escribir la solución, si se encontró alguna solución factible
if m.SolCount > 0:
    # recuperar los valores de las variables
    vx = m.getAttr('x', x)
    print('\nRutas óptimas:')
    for i,j in A:
        if vx[i,j] >= 0.99:
            print('{} -> {}'.format(i, j))

Para este problema, es más informativo escribir la respuesta como una *lista de rutas*, que nos indique para cada ruta el orden en el que se visitan los nodos clientes:

In [ ]:
# crear lista con arcos seleccionados en la solución
vx = m.getAttr('x', x)
L = gp.tuplelist([(i,j) for (i,j) in A if vx[i,j]>=0.9])

# recuperar el número de rutas
n_rutas = len(L.select(0,'*'))

# recuperar cada ruta como un ordenamiento de los nodos que empieza y termina en el depósito
rutas = []
for i in range(n_rutas):
    T = [0]
    # nodo actual:
    i = 0
    while True:
        # determinar un sucesor de i
        (i,j) = L.select(i,'*').pop(0)
        T.append(j)
        i = j
        if i==0:
            break
    rutas.append(T)

print ('*** Rutas óptimas:')
for i in range(len(rutas)):
    print('r{}: {}'.format(i+1, rutas[i]))


Finalmente, es posible graficar la solución utilizando el módulo `matplotlib`:

In [ ]:
import matplotlib.pyplot as plt

def dibujarRutas(coordx, coordy, rutas):
    plt.figure(figsize=(10,10))
    for tour in rutas:
        Tx = [coordx[i] for i in tour]
        Ty = [coordy[i] for i in tour]
        plt.plot(Tx[:-1], Ty[:-1], 'ro')
        for i in range(len(tour)-1):
            s='{}'.format(tour[i])
            plt.text(Tx[i],Ty[i]+1,s)
            plt.arrow(Tx[i], Ty[i], Tx[i+1]-Tx[i], Ty[i+1]-Ty[i], color='blue', 
                  length_includes_head=True, width=0.1, head_width=1)
    plt.show()

# Graficar el tour
dibujarRutas(coordx, coordy, rutas) 

## Código completo

Se reproduce a continuación el código completo del modelo anterior.

In [ ]:
# Implementación de modelos lineales enteros
# Problema del agente viajero asimétrico múltiple (k-ATSP)

# Luis M. Torres (EPN 2025)

import gurobipy as gp
from gurobipy import GRB
import random as rm
import matplotlib.pyplot as plt
import math 

def dibujarRutas(coordx, coordy, rutas):
    plt.figure(figsize=(10,10))
    for tour in rutas:
        Tx = [coordx[i] for i in tour]
        Ty = [coordy[i] for i in tour]
        plt.plot(Tx[:-1], Ty[:-1], 'ro')
        for i in range(len(tour)-1):
            s='{}'.format(tour[i])
            plt.text(Tx[i],Ty[i]+1,s)
            plt.arrow(Tx[i], Ty[i], Tx[i+1]-Tx[i], Ty[i+1]-Ty[i], color='blue', 
                  length_includes_head=True, width=0.1, head_width=1)
    plt.show()


# número de nodos del grafo
n = 40

# número de rutas en la solución
k = 4

# nodos del grafo
V = gp.tuplelist(range(0,n+1))

# nodos sin el depósito
N = gp.tuplelist([i for i in V if i!=0])

# posiciones de los nodos en un plano euclideano entre (0,0) y (100, 100)
coordx={i : rm.randint(0,100) for i in N}
coordy={i : rm.randint(0,100) for i in N}
# el depósito está en el centro
coordx[0] = 50
coordy[0] = 50

# los costos son las distancias eculideanas
c = gp.tupledict({
    (i,j) : math.sqrt((coordx[i] - coordx[j])**2 + (coordy[i] - coordy[j])**2)
    for i in V for j in V if i!=j
})

A = c.keys()

try:
    # crear el objeto modelo
    m = gp.Model('k-atsp')

    # crear las variables de selección de arcos
    x = m.addVars(A, name="x", vtype=GRB.BINARY)

    # crear las variables de ordenamiento de nodos
    u = m.addVars(N, name="u", lb=1, ub=n)

    # crear la función objetivo
    m.setObjective(x.prod(c,'*'), GRB.MINIMIZE)

    ### restricciones
    # restricciones de grado saliente clientes
    m.addConstrs((x.sum(i,'*') == 1 for i in N), "g_saliente")

    # restricciones de grado entrante clientes
    m.addConstrs((x.sum('*', i) == 1 for i in N), "g_entrante")

    # restricción de grado saliente depósito
    m.addConstr(x.sum(0,'*') == k, "d_saliente")

    # restricción de grado entrante depósito
    m.addConstr(x.sum('*', 0) == k, "d_entrante")

    # restricciones de ordenamiento de nodos (eliminación de subciclos)
    m.addConstrs((u[j] >= u[i] + 1 - n*(1- x[i,j]) for i in N for j in N if i!=j), 
                 "ordenamiento")

    # terminar al alcanzar un Gap del 10%
    # m.Params.MIPGap = 0.1

    # terminar luego de 180 segundos
    m.Params.TimeLimit = 180

    # resolver el modelo
    m.optimize()

    # escribir la solución, si se encontró alguna solución factible
    if m.SolCount > 0:
        # crear lista con arcos seleccionados en la solución
        vx = m.getAttr('x', x)
        L = gp.tuplelist([(i,j) for (i,j) in A if vx[i,j]>=0.9])
        # recuperar el número de rutas
        n_rutas = len(L.select(0,'*'))
        # recuperar cada ruta como un ordenamiento de los nodos que empieza y termina en el depósito
        rutas = []
        for i in range(n_rutas):
            T = [0]
            # nodo actual:
            i = 0
            while True:
                # determinar un sucesor de i
                (i,j) = L.select(i,'*').pop(0)
                T.append(j)
                i = j
                if i==0:
                    break
            rutas.append(T)

        print ('*** Rutas óptimas:')
        for i in range(len(rutas)):
            print('r{}: {}'.format(i+1, rutas[i]))
        
        # dibujar las rutas de la solución
        dibujarRutas(coordx, coordy, rutas) 

except GurobiError as e:
    print('Se produjo un error de Gurobi: código: ' + str(e.errno) + ": " + str(e))

except AttributeError:
    print('Se produjo un error de atributo')